<a href="https://colab.research.google.com/github/anablima/MBA_DataEngineer/blob/main/Bootcamp_Data_Engineering_Exercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Temos duas bases de dados pessoais (fictícios)


*   cadastro.csv (dados de cadastro)
*   dados_medicos.csv (registros médicos)

Temos que unificar as duas bases e criar uma nova base agrupada

Código localizado em https://github.com/renzoziegler/data_engineering_bootcamp


Exercício 1:


*   Importar o Pandas
*   Instalar e importar a biblioteca fuzzywuzzy (será importante mais pra frente)
*   Ler os arquivos CSV fornecidos
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv
*   Realizar uma análise exploratória dos dados



In [ ]:
# Instalando e importando libs necessárias
!pip install fuzzywuzzy
import fuzzywuzzy
import pandas as pd
import numpy as np

In [ ]:
# Importando dados de cadastro
cad = pd.read_csv("https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv", sep=",")
data_med = pd.read_csv("https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv", sep=",")

Para análise exploratória, vamos entender as bases de dados: seu tamanho, quais dados estão disponíveis, qual o formato deles, se estão completos ou não.....

In [ ]:
# Exibindo primeiras linhas do dataframe
cad.head()

In [ ]:
# Obtendo informações sobre o DataFrame
cad.info()

In [ ]:
# Resumo das colunas numéricas
cad.describe()

In [ ]:
# Verificando se há valores nulos
cad.isnull().sum()

In [ ]:
# Exibindo primeiras linhas do dataframe
data_med.head(100)

In [ ]:
# Obtendo informações sobre o DataFrame
data_med.info()

In [ ]:
# Resumo das colunas numéricas
data_med.describe()

In [ ]:
# Verificando a quantidade de valores únicos em cada coluna
data_med.nunique()

In [ ]:
# Verificando se há valores nulos
data_med.isnull().sum()

Exercício 2: Pré-processamento
*   Quais tarefas vocês listam como necessárias?
 *  Padronização de campos
 *  Substituir valores
 *  Retirar caracteres especiais
*   Listar e executar!

Tratamento Dataframe cad - Dados de Cadastro

In [ ]:
# Padronização de campos - Garantindo tamanho do cpf
cad['cpf'] = cad['cpf'].astype(str).str.zfill(11)
cad['cpf'] = cad['cpf'].astype(str) # Convertendo a coluna 'cpf' para string
cad.head()

In [ ]:
# Padronização de campos
# 1. Convertendo a coluna 'data_nasc' para datetime
cad['data_nasc'] = pd.to_datetime(cad['data_nasc'], format='%d-%m-%Y', errors='coerce')
cad.head()

In [ ]:
# 2. Formatando a coluna 'data_nasc' para o formato desejado
cad['data_nasc'] = cad['data_nasc'].dt.strftime('%d/%m/%Y')
cad.head()

In [ ]:
# Substituição de valores
# Remove o prefixo e o DDD do campo 'celular'
cad['celular'] = cad['celular'].astype(str).str[-9:]

# Aplica uma função lambda para modificar os números de celular que começam com '8'
cad['celular'] = cad['celular'].apply(lambda x: '9' + x[1:] if x.startswith('8') else x)

# Remove o prefixo e o DDD do campo 'telefone_fixo'
cad['telefone_fixo'] = cad['telefone_fixo'].astype(str).str[-8:]

cad.head(100)

Tratamento Dataframe data_med - Dados Médicos

In [ ]:
# Padronização de campos
# Removendo zeros à esquerda e convertendo para string
data_med['cpf'] = data_med['cpf'].astype(str).str.lstrip('0')

# Adicionando zeros à esquerda para completar 11 dígitos
data_med['cpf'] = data_med['cpf'].str.zfill(11)

data_med.head(100)

In [ ]:
# 1. Convertendo a coluna 'data_nasc' para datetime
data_med['data_nasc'] = pd.to_datetime(cad['data_nasc'], format='%d-%m-%Y', errors='coerce')

In [ ]:
# 2. Formatando a coluna 'data_nasc' para o formato desejado
data_med['data_nasc'] = data_med['data_nasc'].dt.strftime('%d/%m/%Y')

data_med.head(100)

In [ ]:
# Identifica datas inválidas e as altera para "Inválido"
for i in range(len(data_med)):
    try:
        pd.to_datetime(data_med.loc[i, 'data_nasc'])
    except:
        data_med.loc[i, 'data_nasc'] = "Inválido"

# Converte a coluna 'data_nasc' para string
data_med['data_nasc'] = data_med['data_nasc'].astype(str)

data_med.head(100)

Exercício 3: Indexação
*    Listar possíveis blocking Keys
*    Definir uma a ser utilizada
*    Criar os blocos
*    Listar os possíveis matches em cada bloco

In [135]:
# Listar possíveis blocking Keys - Dataframe cad
print("CPF: ", cad['cpf'].nunique())
print("Nome: ", cad['nome'].nunique())
print("Celular: ", cad['celular'].nunique())

CPF:  391
Nome:  393
Celular:  391


In [143]:
# Criando os blocos
blocos_cad = cad.groupby('cpf')

# Listando os possíveis matches em cada bloco
for _, bloco in blocos_cad:
    print(bloco)

                      nome   data_nasc        cpf                      mae  \
302  Levi Gael Jorge Viana  04-10-1951  200456628  Antônia Silvana Sandra    

       celular                                email telefone_fixo  
302  996272230  levigaeljorgeviana-79@imobideal.com      37072129  
                             nome   data_nasc        cpf  \
287  Kevin Anthony Thales Barbosa  11-03-1998  267627904   

                          mae    celular  \
287  Rafaela Marlene Larissa   981340931   

                                                email telefone_fixo  
287  kevinanthonythalesbarbosa_@senhorasdaarte.com.br      25938168  
                                 nome   data_nasc         cpf           mae  \
368  Cecília Juliana Larissa Assunção  15-12-1939  1179619307  Luna Tereza    

       celular                                              email  \
368  987277839  ceciliajulianalarissaassuncao_@2emesconstrutor...   

    telefone_fixo  
368      38224033  
                   

In [140]:
# Listar possíveis blocking Keys - Dataframe data_med
print("CPF: ", data_med['cpf'].nunique()) # CPF
print("Nome: ", data_med['nome'].nunique()) # Nome (apesar de ter maior probabilidade de erros de digitação)

CPF:  302
Nome:  368


In [142]:
# Criando os blocos
blocos_data_med = data_med.groupby('cpf')

# Listando os possíveis matches em cada bloco
for _, bloco in blocos_data_med:
    print(bloco)

                              nome   data_nasc          cpf tipo_sanguineo  \
8            Benício Iago Silveira  17/05/1993  00000000000             O+   
19           Betina Elaine Drumond  20/03/1949  00000000000            AB-   
21   Sandra Vitória Lavínia Barros  12/07/1973  00000000000            AB+   
28       Pedro Marcos Tiago Galvão  09/09/1968  00000000000             B-   
42          Jennifer A Sabrina Paz  03/12/1956  00000000000             O-   
..                             ...         ...          ...            ...   
343            Isadora Luzia Brito  27/09/1980  00000000000             O-   
350    Bernardo R Roberto Silveira  12/12/1968  00000000000             B+   
356        Silvana Joana Conceicão  19/03/1979  00000000000            AB+   
365        Rayssa D Priscila Alves  22/04/1978  00000000000             O-   
366           Emilly Sarah Drumond  24/02/1979  00000000000            AB-   

     peso altura  
8      97   1,72  
19     74   1,58  
21    

Exercício 4: Comparação par a par
*    Desenvolver um algoritmo de comparação de cada match potencial do exercício anterior
*    Definir uma nota de similaridade para cada comparação
 *   Para cada parâmetro

Dataframe cad

In [153]:
from fuzzywuzzy import fuzz
# Comparando cada match potencial do dataframe cad
for _, bloco in blocos_cad:
  for i in range(len(bloco)):
    for j in range(i + 1, len(bloco)):
      registro1 = bloco.iloc[i]
      registro2 = bloco.iloc[j]

      # Comparando nomes
      similaridade_nome = fuzzywuzzy.fuzz.ratio(registro1['nome'], registro2['nome'])

      # Comparando datas de nascimento
      similaridade_data_nasc = fuzzywuzzy.fuzz.ratio(registro1['data_nasc'], registro2['data_nasc'])

      # Comparando celulares
      similaridade_celular = fuzzywuzzy.fuzz.ratio(registro1['celular'], registro2['celular'])

      # Calculando uma nota de similaridade geral (pode ser ajustado conforme a necessidade)
      nota_similaridade = (similaridade_nome + similaridade_data_nasc + similaridade_celular) / 3

      print(f"Comparando registros {i+1} e {j+1} do bloco:")
      print(f"Similaridade nome: {similaridade_nome}")
      print(f"Similaridade data de nascimento: {similaridade_data_nasc}")
      print(f"Similaridade celular: {similaridade_celular}")
      print(f"Nota de similaridade geral: {nota_similaridade}")
      print("----")

Comparando registros 1 e 2 do bloco:
Similaridade nome: 77
Similaridade data de nascimento: 100
Similaridade celular: 100
Nota de similaridade geral: 92.33333333333333
----
Comparando registros 1 e 3 do bloco:
Similaridade nome: 76
Similaridade data de nascimento: 30
Similaridade celular: 100
Nota de similaridade geral: 68.66666666666667
----
Comparando registros 2 e 3 do bloco:
Similaridade nome: 85
Similaridade data de nascimento: 30
Similaridade celular: 100
Nota de similaridade geral: 71.66666666666667
----


In [152]:
from fuzzywuzzy import fuzz

def comparar_matches(bloco):
  """
  Compara cada par de registros em um bloco e calcula a similaridade para cada parâmetro.

  Args:
    bloco: Um DataFrame contendo os registros a serem comparados.

  Returns:
    Um DataFrame com as similaridades calculadas.
  """

  resultados = []
  for i in range(len(bloco)):
    for j in range(i + 1, len(bloco)):
      registro1 = bloco.iloc[i]
      registro2 = bloco.iloc[j]

      similaridade_nome = fuzz.ratio(registro1['nome'], registro2['nome'])
      similaridade_data_nasc = fuzz.ratio(registro1['data_nasc'], registro2['data_nasc'])
      similaridade_celular = fuzz.ratio(registro1['celular'], registro2['celular'])
      similaridade_telefone_fixo = fuzz.ratio(registro1['telefone_fixo'], registro2['telefone_fixo'])
      similaridade_email = fuzz.ratio(registro1['email'], registro2['email'])

      resultados.append({
          'cpf1': registro1['cpf'],
          'cpf2': registro2['cpf'],
          'similaridade_nome': similaridade_nome,
          'similaridade_data_nasc': similaridade_data_nasc,
          'similaridade_celular': similaridade_celular,
          'similaridade_telefone_fixo': similaridade_telefone_fixo,
          'similaridade_email': similaridade_email
      })

  return pd.DataFrame(resultados)

# Aplica a função a cada bloco do DataFrame 'cad'
resultados_comparacao = blocos_cad.apply(comparar_matches).reset_index(drop=True)

resultados_comparacao.head()

,cpf1,cpf2,similaridade_nome,similaridade_data_nasc,similaridade_celular,similaridade_telefone_fixo,similaridade_email
0,5.876242e+10,5.876242e+10,77.0,100.0,100.0,100.0,100.0
1,5.876242e+10,5.876242e+10,76.0,30.0,100.0,100.0,100.0
2,5.876242e+10,5.876242e+10,85.0,30.0,100.0,100.0,100.0


Dataframe data_med

In [155]:
from fuzzywuzzy import fuzz
# Comparando cada match potencial do dataframe cad
for _, bloco in blocos_data_med:
  for i in range(len(bloco)):
    for j in range(i + 1, len(bloco)):
      registro1 = bloco.iloc[i]
      registro2 = bloco.iloc[j]

      # Comparando nomes
      similaridade_nome = fuzzywuzzy.fuzz.ratio(registro1['nome'], registro2['nome'])

      # Comparando datas de nascimento
      similaridade_data_nasc = fuzzywuzzy.fuzz.ratio(registro1['data_nasc'], registro2['data_nasc'])

      # Calculando uma nota de similaridade geral (pode ser ajustado conforme a necessidade)
      nota_similaridade = (similaridade_nome + similaridade_data_nasc) / 2

      print(f"Comparando registros {i+1} e {j+1} do bloco:")
      print(f"Similaridade nome: {similaridade_nome}")
      print(f"Similaridade data de nascimento: {similaridade_data_nasc}")
      print(f"Nota de similaridade geral: {nota_similaridade}")
      print("----")

A saída de streaming foi truncada nas últimas 5000 linhas.
Comparando registros 20 e 59 do bloco:
Similaridade nome: 26
Similaridade data de nascimento: 50
Nota de similaridade geral: 38.0
----
Comparando registros 20 e 60 do bloco:
Similaridade nome: 27
Similaridade data de nascimento: 60
Nota de similaridade geral: 43.5
----
Comparando registros 20 e 61 do bloco:
Similaridade nome: 34
Similaridade data de nascimento: 50
Nota de similaridade geral: 42.0
----
Comparando registros 20 e 62 do bloco:
Similaridade nome: 24
Similaridade data de nascimento: 60
Nota de similaridade geral: 42.0
----
Comparando registros 20 e 63 do bloco:
Similaridade nome: 18
Similaridade data de nascimento: 50
Nota de similaridade geral: 34.0
----
Comparando registros 20 e 64 do bloco:
Similaridade nome: 22
Similaridade data de nascimento: 40
Nota de similaridade geral: 31.0
----
Comparando registros 20 e 65 do bloco:
Similaridade nome: 14
Similaridade data de nascimento: 50
Nota de similaridade geral: 32.0
-

Exercício 5: Classificação
*    Classificar cada par, baseado num threshold
*    Definir um peso para cada parâmetro

In [ ]:
#Seu código aqui

Exercício 6: Avaliação
*     Buscar uma amostra das classificações (de 10 a 20 registros)
*     Classificar cada uma em TP, TN, FP, FN
*     Calcular acurácia, precisão e recall

In [ ]:
#Seu código aqui